In [1]:
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random
import datetime
from math import ceil

from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings('ignore')

In [2]:
def get_memory_usage():
    return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2) 
        
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

In [3]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
## Merging by concat to not lose dtypes
def merge_by_concat(df1, df2, merge_on):
    merged_gf = df1[merge_on]
    merged_gf = merged_gf.merge(df2, on=merge_on, how='left')
    new_columns = [col for col in list(merged_gf) if col not in merge_on]
    df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
    return df1

In [5]:
TARGET = 'sales'         # Our main target
END_TRAIN = 1913         # Last day in train set
MAIN_INDEX = ['id','d']  # We can identify item by these columns

In [6]:
print('Load Main Data')
# Here are reafing all our data 
# without any limitations and dtype modification
train_df = pd.read_csv('sales_train_validation.csv')
prices_df = pd.read_csv('sell_prices.csv')
calendar_df = pd.read_csv('calendar.csv')

Load Main Data


In [7]:
print('Create Grid')

# We can tranform horizontal representation 
# to vertical "view"
# Our "index" will be 'id','item_id','dept_id','cat_id','store_id','state_id'
# and labels are 'd_' coulmns

index_columns = ['id','item_id','dept_id','cat_id','store_id','state_id']
grid_df = pd.melt(train_df, 
                  id_vars = index_columns, 
                  var_name = 'd', 
                  value_name = TARGET)

# If we look on train_df we see that 
# we don't have a lot of traning rows
# but each day can provide more train data
print('Train rows:', len(train_df), len(grid_df))

# To be able to make predictions
# we need to add "test set" to our grid
add_grid = pd.DataFrame()
for i in range(1,29):
    temp_df = train_df[index_columns]
    temp_df = temp_df.drop_duplicates()
    temp_df['d'] = 'd_'+ str(END_TRAIN+i)
    temp_df[TARGET] = np.nan
    add_grid = pd.concat([add_grid,temp_df])

grid_df = pd.concat([grid_df,add_grid])
grid_df = grid_df.reset_index(drop=True)

# Remove some temoprary DFs
del temp_df, add_grid

# We will not need original train_df
# anymore and can remove it
del train_df

# You don't have to use df = df construction
# you can use inplace=True instead.
# like this
# grid_df.reset_index(drop=True, inplace=True)

# Let's check our memory usage
print("{:>20}: {:>8}".format('Original grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

# We can free some memory 
# by converting "strings" to categorical
# it will not affect merging and 
# we will not lose any valuable data
for col in index_columns:
    grid_df[col] = grid_df[col].astype('category')

# Let's check again memory usage
print("{:>20}: {:>8}".format('Reduced grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

Create Grid
Train rows: 30490 58327370
    Original grid_df:   3.5GiB
     Reduced grid_df:   1.3GiB


In [8]:
grid_df.head() #1-1913:train

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0


In [9]:
grid_df.tail() #1914-1941:test

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
59181085,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1941,NaN
59181086,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1941,NaN
59181087,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,NaN
59181088,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1941,NaN
59181089,FOODS_3_827_WI_3_validation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1941,NaN


In [10]:
prices_df[(prices_df['store_id']=='CA_1') & (prices_df['item_id']=='FOODS_1_001')].head()

,store_id,item_id,wm_yr_wk,sell_price
368746,CA_1,FOODS_1_001,11101,2.0
368747,CA_1,FOODS_1_001,11102,2.0
368748,CA_1,FOODS_1_001,11103,2.0
368749,CA_1,FOODS_1_001,11104,2.0
368750,CA_1,FOODS_1_001,11105,2.0


In [11]:
print('Release week')

# It seems that leadings zero values
# in each train_df item row
# are not real 0 sales but mean
# absence for the item in the store
# we can safe some memory by removing
# such zeros

# Prices are set by week
# so it we will have not very accurate release week 
release_df = prices_df.groupby(['store_id','item_id'])['wm_yr_wk'].agg(['min']).reset_index()
release_df.columns = ['store_id','item_id','release']

Release week


In [12]:
release_df.head() # release: the first wm_yr_wk has price

,store_id,item_id,release
0,CA_1,FOODS_1_001,11101
1,CA_1,FOODS_1_002,11101
2,CA_1,FOODS_1_003,11101
3,CA_1,FOODS_1_004,11206
4,CA_1,FOODS_1_005,11101


In [13]:
# Now we can merge release_df
grid_df = merge_by_concat(grid_df, release_df, ['store_id','item_id'])
del release_df

In [14]:
grid_df[(grid_df['store_id']=='CA_1') & (grid_df['item_id']=='FOODS_1_001')].head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,release
1612,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1,3.0,11101
32102,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_2,0.0,11101
62592,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_3,0.0,11101
93082,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_4,1.0,11101
123572,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_5,4.0,11101


In [15]:
# We want to remove some "zeros" rows
# from grid_df 
# to do it we need wm_yr_wk column
# let's merge partly calendar_df to have it
grid_df = merge_by_concat(grid_df, calendar_df[['wm_yr_wk','d']], ['d'])

In [16]:
grid_df[(grid_df['store_id']=='CA_1') & (grid_df['item_id']=='FOODS_1_001')].head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,release,wm_yr_wk
1612,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1,3.0,11101,11101
32102,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_2,0.0,11101,11101
62592,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_3,0.0,11101,11101
93082,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_4,1.0,11101,11101
123572,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_5,4.0,11101,11101


In [17]:
grid_df.shape

(59181090, 10)

In [18]:
# Now we can cutoff some rows 
# and safe memory 
grid_df = grid_df[grid_df['wm_yr_wk']>=grid_df['release']]
grid_df = grid_df.reset_index(drop=True)

# Let's check our memory usage
print("{:>20}: {:>8}".format('Original grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

    Original grid_df:   1.7GiB


In [19]:
grid_df[(grid_df['store_id']=='CA_1') & (grid_df['item_id']=='FOODS_1_001')].head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,release,wm_yr_wk
576,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1,3.0,11101,11101
11508,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_2,0.0,11101,11101
22440,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_3,0.0,11101,11101
33372,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_4,1.0,11101,11101
44304,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_5,4.0,11101,11101


In [20]:
grid_df.shape

(46881677, 10)

In [21]:
# Should we keep release week 
# as one of the features?
# Only good CV can give the answer.
# Let's minify the release values.
# Min transformation will not help here 
# as int16 -> Integer (-32768 to 32767)
# and our grid_df['release'].max() serves for int16
# but we have have an idea how to transform 
# other columns in case we will need it
grid_df['release'] = grid_df['release'] - grid_df['release'].min()
grid_df['release'] = grid_df['release'].astype(np.int16)

# Let's check again memory usage
print("{:>20}: {:>8}".format('Reduced grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

     Reduced grid_df:   1.5GiB


In [22]:
grid_df[(grid_df['store_id']=='CA_1') & (grid_df['item_id']=='FOODS_1_001')].head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,release,wm_yr_wk
576,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1,3.0,0,11101
11508,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_2,0.0,0,11101
22440,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_3,0.0,0,11101
33372,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_4,1.0,0,11101
44304,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_5,4.0,0,11101


In [23]:
print('Save Part 1')

# We have our BASE grid ready
# and can save it as pickle file
# for future use (model training)
grid_df.to_pickle('grid_part_1.pkl')

print('Size:', grid_df.shape)

Save Part 1
Size: (46881677, 10)


In [24]:
# add price feature
prices_df.head(10)

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26
5,CA_1,HOBBIES_1_001,11330,8.26
6,CA_1,HOBBIES_1_001,11331,8.26
7,CA_1,HOBBIES_1_001,11332,8.26
8,CA_1,HOBBIES_1_001,11333,8.26
9,CA_1,HOBBIES_1_001,11334,8.26


In [25]:
print('Prices')

# We can do some basic aggregations
prices_df['price_max'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('max')
prices_df['price_min'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('min')
prices_df['price_std'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('std')
prices_df['price_mean'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('mean')

# and do price normalization (min/max scaling)
prices_df['price_norm'] = prices_df['sell_price']/prices_df['price_max']


Prices


# #price_rank_dept

In [26]:
# price_rank_dept
prices_df["dept"]=prices_df["item_id"].apply(lambda x: x.split('_'))
prices_df["dept"]=prices_df["dept"].apply(lambda x: str(x[0:2]))

cat=prices_df["dept"].value_counts().index

a=pd.DataFrame()
k=10
for i in range(len(cat))    :
    temp=pd.cut(prices_df[prices_df["dept"]==cat[i]]['sell_price'],k,labels = range(k))
    a=pd.concat([a,temp])
a.columns=["price_rank_dept"]
a.sort_index(inplace=True)

prices_df=pd.concat([prices_df,a],axis=1)
prices_df["price_rank_dept"]=prices_df["price_rank_dept"].astype("category")
prices_df.drop(["dept"],axis=1,inplace=True)
del a,cat,k
gc.collect()

42

In [27]:
prices_df.head()

,store_id,item_id,wm_yr_wk,sell_price,price_max,price_min,price_std,price_mean,price_norm,price_rank_dept
0,CA_1,HOBBIES_1_001,11325,9.58,9.58,8.26,0.152139,8.285714,1.000000,3
1,CA_1,HOBBIES_1_001,11326,9.58,9.58,8.26,0.152139,8.285714,1.000000,3
2,CA_1,HOBBIES_1_001,11327,8.26,9.58,8.26,0.152139,8.285714,0.862213,2
3,CA_1,HOBBIES_1_001,11328,8.26,9.58,8.26,0.152139,8.285714,0.862213,2
4,CA_1,HOBBIES_1_001,11329,8.26,9.58,8.26,0.152139,8.285714,0.862213,2


In [28]:
# Some items are can be inflation dependent
# and some items are very "stable"
# check how many unique values each group has
prices_df['price_nunique'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('nunique')
prices_df['item_nunique'] = prices_df.groupby(['store_id','sell_price'])['item_id'].transform('nunique')

In [29]:
prices_df.head(10)

,store_id,item_id,wm_yr_wk,sell_price,price_max,price_min,price_std,price_mean,price_norm,price_rank_dept,price_nunique,item_nunique
0,CA_1,HOBBIES_1_001,11325,9.58,9.58,8.26,0.152139,8.285714,1.000000,3,3.0,3
1,CA_1,HOBBIES_1_001,11326,9.58,9.58,8.26,0.152139,8.285714,1.000000,3,3.0,3
2,CA_1,HOBBIES_1_001,11327,8.26,9.58,8.26,0.152139,8.285714,0.862213,2,3.0,5
3,CA_1,HOBBIES_1_001,11328,8.26,9.58,8.26,0.152139,8.285714,0.862213,2,3.0,5
4,CA_1,HOBBIES_1_001,11329,8.26,9.58,8.26,0.152139,8.285714,0.862213,2,3.0,5
5,CA_1,HOBBIES_1_001,11330,8.26,9.58,8.26,0.152139,8.285714,0.862213,2,3.0,5
6,CA_1,HOBBIES_1_001,11331,8.26,9.58,8.26,0.152139,8.285714,0.862213,2,3.0,5
7,CA_1,HOBBIES_1_001,11332,8.26,9.58,8.26,0.152139,8.285714,0.862213,2,3.0,5
8,CA_1,HOBBIES_1_001,11333,8.26,9.58,8.26,0.152139,8.285714,0.862213,2,3.0,5
9,CA_1,HOBBIES_1_001,11334,8.26,9.58,8.26,0.152139,8.285714,0.862213,2,3.0,5


In [30]:
# I would like some "rolling" aggregations
# but would like months and years as "window"
calendar_prices = calendar_df[['wm_yr_wk','month','year']]
calendar_prices = calendar_prices.drop_duplicates(subset=['wm_yr_wk'])
prices_df = prices_df.merge(calendar_prices[['wm_yr_wk','month','year']], on=['wm_yr_wk'], how='left')
del calendar_prices

# Now we can add price "momentum" (some sort of)
# Shifted by week 
# by month mean
# by year mean
prices_df['price_momentum'] = prices_df['sell_price']/prices_df.groupby(['store_id','item_id'])['sell_price'].transform(lambda x: x.shift(1))
prices_df['price_momentum_m'] = prices_df['sell_price']/prices_df.groupby(['store_id','item_id','month'])['sell_price'].transform('mean')
prices_df['price_momentum_y'] = prices_df['sell_price']/prices_df.groupby(['store_id','item_id','year'])['sell_price'].transform('mean')

# weather-box

In [31]:
## https://zh.weatherspark.com/y/
# 'temperature_high' 
# 'temperature_low' 
# temperature: 
# 极冷 0
# 冰冻 1
# 很冷 2
# 寒冷 3
# 凉爽 4
# 舒适 5
# 暖和 6
# 热 7
# 热 8


In [32]:
ca_mask=(prices_df["store_id"]=="CA_1")|(prices_df["store_id"]=="CA_2")|(prices_df["store_id"]=="CA_3")|(prices_df["store_id"]=="CA_4")
tx_mask=(prices_df["store_id"]=="TX_1")|(prices_df["store_id"]=="TX_2")|(prices_df["store_id"]=="TX_3")
wi_mask=(prices_df["store_id"]=="WI_1")|(prices_df["store_id"]=="WI_2")|(prices_df["store_id"]=="WI_3")


In [33]:


#WEATHER PARMS
prices_df['temperature_high']=np.nan
prices_df['temperature_con']=np.nan
prices_df['rainfall_m']=np.nan
prices_df['snow_m']=np.nan

## wi_mask 
# temperature_con

prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==1))].index,'temperature_con']=1
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==2))].index,'temperature_con']=2
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==3))].index,'temperature_con']=2.5
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==4))].index,'temperature_con']=3.5
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==5))].index,'temperature_con']=4.5
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==6))].index,'temperature_con']=5.5
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==7))].index,'temperature_con']=6
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==8))].index,'temperature_con']=6
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==9))].index,'temperature_con']=5
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==10))].index,'temperature_con']=4
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==11))].index,'temperature_con']=3
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==12))].index,'temperature_con']=2

# temperature_high

prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==1))].index,'temperature_high']=0.5
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==2))].index,'temperature_high']=2
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==3))].index,'temperature_high']=6
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==4))].index,'temperature_high']=12
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==5))].index,'temperature_high']=18
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==6))].index,'temperature_high']=(21+27)/2
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==7))].index,'temperature_high']=27
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==8))].index,'temperature_high']=25
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==9))].index,'temperature_high']=22
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==10))].index,'temperature_high']=(17+12)/2
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==11))].index,'temperature_high']=(12+4)/2
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==12))].index,'temperature_high']=(4+0)/2

# rainfall_m

prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==1))].index,'rainfall_m']=16
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==2))].index,'rainfall_m']=21
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==3))].index,'rainfall_m']=42
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==4))].index,'rainfall_m']=72
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==5))].index,'rainfall_m']=80
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==6))].index,'rainfall_m']=86
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==7))].index,'rainfall_m']=80
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==8))].index,'rainfall_m']=82
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==9))].index,'rainfall_m']=76
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==10))].index,'rainfall_m']=63
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==11))].index,'rainfall_m']=50
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==12))].index,'rainfall_m']=31

# snow_m

prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==1))].index,'snow_m']=11
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==2))].index,'snow_m']=10
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==3))].index,'snow_m']=5
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==4))].index,'snow_m']=2
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==5))].index,'snow_m']=0
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==6))].index,'snow_m']=0
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==7))].index,'snow_m']=0
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==8))].index,'snow_m']=0
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==9))].index,'snow_m']=0
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==10))].index,'snow_m']=0
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==11))].index,'snow_m']=3
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==12))].index,'snow_m']=9





In [34]:
## ca_mask 
# temperature_con

prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==1))].index,'temperature_con']=5
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==2))].index,'temperature_con']=5
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==3))].index,'temperature_con']=5
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==4))].index,'temperature_con']=5
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==5))].index,'temperature_con']=5
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==6))].index,'temperature_con']=6
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==7))].index,'temperature_con']=6
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==8))].index,'temperature_con']=6
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==9))].index,'temperature_con']=6
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==10))].index,'temperature_con']=6
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==11))].index,'temperature_con']=5
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==12))].index,'temperature_con']=5

# temperature_high

prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==1))].index,'temperature_high']=20
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==2))].index,'temperature_high']=20
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==3))].index,'temperature_high']=21
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==4))].index,'temperature_high']=22.5
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==5))].index,'temperature_high']=23.5
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==6))].index,'temperature_high']=25.5
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==7))].index,'temperature_high']=28
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==8))].index,'temperature_high']=29.5
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==9))].index,'temperature_high']=28.5
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==10))].index,'temperature_high']=25.5
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==11))].index,'temperature_high']=22
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==12))].index,'temperature_high']=20

# rainfall_m

prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==1))].index,'rainfall_m']=69
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==2))].index,'rainfall_m']=79
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==3))].index,'rainfall_m']=50
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==4))].index,'rainfall_m']=20
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==5))].index,'rainfall_m']=5
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==6))].index,'rainfall_m']=1.5
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==7))].index,'rainfall_m']=0
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==8))].index,'rainfall_m']=1
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==9))].index,'rainfall_m']=4
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==10))].index,'rainfall_m']=12
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==11))].index,'rainfall_m']=29
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==12))].index,'rainfall_m']=52

# snow_m

prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==1))].index,'snow_m']=0
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==2))].index,'snow_m']=0
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==3))].index,'snow_m']=0
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==4))].index,'snow_m']=0
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==5))].index,'snow_m']=0
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==6))].index,'snow_m']=0
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==7))].index,'snow_m']=0
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==8))].index,'snow_m']=0
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==9))].index,'snow_m']=0
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==10))].index,'snow_m']=0
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==11))].index,'snow_m']=0
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==12))].index,'snow_m']=0

In [35]:

## tx_mask 
# temperature_con(气候概要)

prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==1))].index,'temperature_con']=4
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==2))].index,'temperature_con']=4
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==3))].index,'temperature_con']=5
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==4))].index,'temperature_con']=6
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==5))].index,'temperature_con']=6
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==6))].index,'temperature_con']=7
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==7))].index,'temperature_con']=8
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==8))].index,'temperature_con']=8
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==9))].index,'temperature_con']=7
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==10))].index,'temperature_con']=6
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==11))].index,'temperature_con']=5
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==12))].index,'temperature_con']=4

# temperature_high

prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==1))].index,'temperature_high']=14
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==2))].index,'temperature_high']=17
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==3))].index,'temperature_high']=21
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==4))].index,'temperature_high']=24.5
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==5))].index,'temperature_high']=28.5
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==6))].index,'temperature_high']=32.5
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==7))].index,'temperature_high']=35
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==8))].index,'temperature_high']=35.5
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==9))].index,'temperature_high']=31
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==10))].index,'temperature_high']=26
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==11))].index,'temperature_high']=19
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==12))].index,'temperature_high']=15

# rainfall_m

prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==1))].index,'rainfall_m']=50
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==2))].index,'rainfall_m']=59
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==3))].index,'rainfall_m']=70
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==4))].index,'rainfall_m']=85
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==5))].index,'rainfall_m']=102
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==6))].index,'rainfall_m']=80
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==7))].index,'rainfall_m']=42
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==8))].index,'rainfall_m']=47
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==9))].index,'rainfall_m']=64
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==10))].index,'rainfall_m']=89
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==11))].index,'rainfall_m']=70
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==12))].index,'rainfall_m']=62

# snow_m

prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==1))].index,'snow_m']=0
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==2))].index,'snow_m']=0
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==3))].index,'snow_m']=0
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==4))].index,'snow_m']=0
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==5))].index,'snow_m']=0
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==6))].index,'snow_m']=0
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==7))].index,'snow_m']=0
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==8))].index,'snow_m']=0
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==9))].index,'snow_m']=0
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==10))].index,'snow_m']=0
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==11))].index,'snow_m']=0
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==12))].index,'snow_m']=0

In [36]:
del prices_df['month'], prices_df['year']

In [37]:
prices_df.head(10)

,store_id,item_id,wm_yr_wk,sell_price,price_max,price_min,price_std,price_mean,price_norm,price_rank_dept,price_nunique,item_nunique,price_momentum,price_momentum_m,price_momentum_y,temperature_high,temperature_con,rainfall_m,snow_m
0,CA_1,HOBBIES_1_001,11325,9.58,9.58,8.26,0.152139,8.285714,1.000000,3,3.0,3,NaN,1.127059,1.145166,28.0,6.0,0.0,0.0
1,CA_1,HOBBIES_1_001,11326,9.58,9.58,8.26,0.152139,8.285714,1.000000,3,3.0,3,1.000000,1.127059,1.145166,28.0,6.0,0.0,0.0
2,CA_1,HOBBIES_1_001,11327,8.26,9.58,8.26,0.152139,8.285714,0.862213,2,3.0,5,0.862213,0.971765,0.987377,28.0,6.0,0.0,0.0
3,CA_1,HOBBIES_1_001,11328,8.26,9.58,8.26,0.152139,8.285714,0.862213,2,3.0,5,1.000000,1.000000,0.987377,29.5,6.0,1.0,0.0
4,CA_1,HOBBIES_1_001,11329,8.26,9.58,8.26,0.152139,8.285714,0.862213,2,3.0,5,1.000000,1.000000,0.987377,29.5,6.0,1.0,0.0
5,CA_1,HOBBIES_1_001,11330,8.26,9.58,8.26,0.152139,8.285714,0.862213,2,3.0,5,1.000000,1.000000,0.987377,29.5,6.0,1.0,0.0
6,CA_1,HOBBIES_1_001,11331,8.26,9.58,8.26,0.152139,8.285714,0.862213,2,3.0,5,1.000000,1.000000,0.987377,29.5,6.0,1.0,0.0
7,CA_1,HOBBIES_1_001,11332,8.26,9.58,8.26,0.152139,8.285714,0.862213,2,3.0,5,1.000000,1.000000,0.987377,29.5,6.0,1.0,0.0
8,CA_1,HOBBIES_1_001,11333,8.26,9.58,8.26,0.152139,8.285714,0.862213,2,3.0,5,1.000000,1.000000,0.987377,28.5,6.0,4.0,0.0
9,CA_1,HOBBIES_1_001,11334,8.26,9.58,8.26,0.152139,8.285714,0.862213,2,3.0,5,1.000000,1.000000,0.987377,28.5,6.0,4.0,0.0


In [38]:
print('Merge prices and save part 2')

# Merge Prices
original_columns = list(grid_df)
grid_df = grid_df.merge(prices_df, on=['store_id','item_id','wm_yr_wk'], how='left')
keep_columns = [col for col in list(grid_df) if col not in original_columns]
grid_df = grid_df[MAIN_INDEX+keep_columns]
grid_df = reduce_mem_usage(grid_df)

# Safe part 2
grid_df.to_pickle('grid_part_2.pkl')
print('Size:', grid_df.shape)

# We don't need prices_df anymore
del prices_df
gc.collect()

Merge prices and save part 2
Mem. usage decreased to 2192.27 Mb (64.7% reduction)
Size: (46881677, 18)


0

In [39]:
grid_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46881677 entries, 0 to 46881676
Data columns (total 18 columns):
id                  category
d                   object
sell_price          float16
price_max           float16
price_min           float16
price_std           float16
price_mean          float16
price_norm          float16
price_rank_dept     category
price_nunique       float16
item_nunique        int16
price_momentum      float16
price_momentum_m    float16
price_momentum_y    float16
temperature_high    float16
temperature_con     float16
rainfall_m          float16
snow_m              float16
dtypes: category(2), float16(14), int16(1), object(1)
memory usage: 2.1+ GB


# calender feature

In [40]:
# add calender feature
calendar_df.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [41]:
calendar_df["event_name_1"].unique()

array([nan, 'SuperBowl', 'ValentinesDay', 'PresidentsDay', 'LentStart',
       'LentWeek2', 'StPatricksDay', 'Purim End', 'OrthodoxEaster',
       'Pesach End', 'Cinco De Mayo', "Mother's day", 'MemorialDay',
       'NBAFinalsStart', 'NBAFinalsEnd', "Father's day",
       'IndependenceDay', 'Ramadan starts', 'Eid al-Fitr', 'LaborDay',
       'ColumbusDay', 'Halloween', 'EidAlAdha', 'VeteransDay',
       'Thanksgiving', 'Christmas', 'Chanukah End', 'NewYear',
       'OrthodoxChristmas', 'MartinLutherKingDay', 'Easter'], dtype=object)

In [42]:
calendar_df["event_type_1"].unique()

array([nan, 'Sporting', 'Cultural', 'National', 'Religious'], dtype=object)

In [43]:
## 上下半月
calendar_df['date']=pd.to_datetime(calendar_df['date'],format="%Y-%m-%d")
# calendar_df['is_first_half_month']=
calendar_df['is_first_half_month']=[ 1 if x.day>=15 else 0  for x in calendar_df['date']]
calendar_df['is_first_half_month']=calendar_df['is_first_half_month'].astype("category")

周级别处理(wday ==1,2 是weekedn)

In [44]:
# calendar_df=calendar_df.reset_index()
calendar_df["weekend"]=[ 1 if x<=2 else 0  for x in calendar_df['wday']]

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,is_first_half_month,weekend
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0,1,1
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0,1,1
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0,1,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0,0,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1,0,0
5,2011-02-03,11101,Thursday,6,2,2011,d_6,NaN,NaN,NaN,NaN,1,1,1,0,0
6,2011-02-04,11101,Friday,7,2,2011,d_7,NaN,NaN,NaN,NaN,1,0,0,0,0
7,2011-02-05,11102,Saturday,1,2,2011,d_8,NaN,NaN,NaN,NaN,1,1,1,0,1
8,2011-02-06,11102,Sunday,2,2,2011,d_9,SuperBowl,Sporting,NaN,NaN,1,1,1,0,1
9,2011-02-07,11102,Monday,3,2,2011,d_10,NaN,NaN,NaN,NaN,1,1,0,0,0


# 节日前后处理 event_index

In [45]:
calendar_df["event_attention_after"]=np.nan
event_index=calendar_df[~calendar_df["event_name_1"].isnull()].index
for i in range(0,8):
    if event_index[-1]+i>=calendar_df.shape[0]:
        event_index=event_index[event_index<(event_index[-1]-i)]
    calendar_df.loc[event_index+i,"event_attention_after"]=7-i
calendar_df.loc[event_index,"event_attention_after"]=7


event_index=calendar_df[~calendar_df["event_name_1"].isnull()].index

In [46]:
calendar_df.head(100)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,is_first_half_month,weekend,event_attention_after
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0,1,1,NaN
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0,1,1,NaN
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0,1,0,NaN
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0,0,0,NaN
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1,0,0,NaN
5,2011-02-03,11101,Thursday,6,2,2011,d_6,NaN,NaN,NaN,NaN,1,1,1,0,0,NaN
6,2011-02-04,11101,Friday,7,2,2011,d_7,NaN,NaN,NaN,NaN,1,0,0,0,0,NaN
7,2011-02-05,11102,Saturday,1,2,2011,d_8,NaN,NaN,NaN,NaN,1,1,1,0,1,NaN
8,2011-02-06,11102,Sunday,2,2,2011,d_9,SuperBowl,Sporting,NaN,NaN,1,1,1,0,1,7.0
9,2011-02-07,11102,Monday,3,2,2011,d_10,NaN,NaN,NaN,NaN,1,1,0,0,0,6.0


In [47]:
calendar_df["event_attention_bef"]=np.nan
for i in range(0,8):
    calendar_df.loc[calendar_df[~calendar_df["event_name_1"].isnull()].index-i,"event_attention_bef"]=i*-1
calendar_df.loc[calendar_df[~calendar_df["event_name_1"].isnull()].index,"event_attention_bef"]=0

In [48]:
calendar_df["event_attention_sum"]=calendar_df["event_attention_bef"]+calendar_df["event_attention_after"]

In [49]:
calendar_df["event_bef_weekend"]=np.nan
event_round_index=calendar_df[~calendar_df["event_attention_bef"].isnull()]
calendar_df.loc[event_round_index[event_round_index.wday<=2].index,"event_bef_weekend"]=1
calendar_df.loc[event_index,"event_bef_weekend"]=0

In [50]:
calendar_df["event_after_weekend"]=np.nan
event_round_index=calendar_df[~calendar_df["event_attention_after"].isnull()]
calendar_df.loc[event_round_index[event_round_index.wday<=2].index,"event_after_weekend"]=1
calendar_df.loc[event_index,"event_after_weekend"]=0

#  NBA

In [51]:
# Lent
# 大斋节亦称“封斋节”，是基督教的斋戒节期。据《新约圣经》载，耶稣于开始传教前在旷野守斋祈祷40昼夜。教会为表示纪念，规定棕枝主日前的40天为此节期。
# 教徒在此期间一般于星期五守大斋和小斋。

# Chanukah End
# 欢庆光明节的人往往会烹制油炸食品，以此象征亮了8天的灯油传奇。特色节日佳肴包括犹太薯饼（latkes）以及有果酱夹心的犹太甜甜圈（sufganya）。



In [52]:
# 赛事数据
calendar_df["NBA"]=np.nan
calendar_df["date"]=pd.to_datetime(calendar_df["date"])
#http://www.uhchina.com/lanqiu/2013-2014nba/
# http://sports.sina.com.cn/nba/playoff1213.html
a=[[2011,5,31],[2011,6,2],[2011,6,5],[2011,6,7],[2011,6,9],[2011,6,12],
   [2012,6,12],[2012,6,14],[2012,6,17],[2012,6,19],[2012,6,21],
   [2013,6,6],[2013,6,9],[2013,6,11],[2013,6,13],[2013,6,16],[2013,6,18],[2013,6,20],
   [2014,6,5],[2014,6,8],[2014,6,10],[2014,6,12],[2014,6,15], 
   [2015,6,4],[2015,6,7],[2015,6,9],[2015,6,11],[2015,6,14], [2015,6,16],
   [2016,6,2],[2016,6,5],[2016,6,8],[2016,6,10],[2016,6,13],[2016,6,16] ,[2016,6,19]
  ]
for i in range(0,len(a)):
    calendar_df.loc[calendar_df[calendar_df["date"]==datetime.datetime(*a[i])].index,"NBA"]=1

In [53]:
calendar_df[calendar_df["NBA"]==1]

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,...,snap_TX,snap_WI,is_first_half_month,weekend,event_attention_after,event_attention_bef,event_attention_sum,event_bef_weekend,event_after_weekend,NBA
122,2011-05-31,11118,Tuesday,4,5,2011,d_123,NBAFinalsStart,Sporting,NaN,...,0,0,1,0,7.0,0.0,7.0,0.0,0.0,1.0
124,2011-06-02,11118,Thursday,6,6,2011,d_125,NaN,NaN,NaN,...,0,1,0,0,4.0,NaN,NaN,NaN,NaN,1.0
127,2011-06-05,11119,Sunday,2,6,2011,d_128,NaN,NaN,NaN,...,1,1,0,1,1.0,-7.0,-6.0,1.0,1.0,1.0
129,2011-06-07,11119,Tuesday,4,6,2011,d_130,NaN,NaN,NaN,...,1,0,0,0,0.0,-5.0,-5.0,NaN,NaN,1.0
131,2011-06-09,11119,Thursday,6,6,2011,d_132,NaN,NaN,NaN,...,1,1,0,0,NaN,-3.0,NaN,NaN,NaN,1.0
134,2011-06-12,11120,Sunday,2,6,2011,d_135,NBAFinalsEnd,Sporting,NaN,...,1,1,0,1,7.0,0.0,7.0,0.0,0.0,1.0
500,2012-06-12,11220,Tuesday,4,6,2012,d_501,NBAFinalsStart,Sporting,NaN,...,1,1,0,0,7.0,0.0,7.0,0.0,0.0,1.0
502,2012-06-14,11220,Thursday,6,6,2012,d_503,NaN,NaN,NaN,...,0,1,0,0,5.0,-7.0,-2.0,NaN,NaN,1.0
505,2012-06-17,11221,Sunday,2,6,2012,d_506,Father's day,Cultural,NaN,...,0,0,1,1,7.0,0.0,7.0,0.0,0.0,1.0
507,2012-06-19,11221,Tuesday,4,6,2012,d_508,NaN,NaN,NaN,...,0,0,1,0,0.0,-2.0,-2.0,NaN,NaN,1.0


In [54]:
grid_df = grid_df[MAIN_INDEX]

# Merge calendar partly
icols = ['date',
         'd',
         #category
         'event_name_1',
         'event_type_1',
         'event_name_2',
         'event_type_2',
         'snap_CA',
         'snap_TX',
         'snap_WI', 
         'is_first_half_month', 
         'event_bef_weekend', 
         'event_after_weekend',
         'NBA',
         
         
         # numerical
        'event_attention_after',
         'event_attention_bef',
         'event_attention_sum',
         


        ]
grid_df = grid_df.merge(calendar_df[icols], on=['d'], how='left')

# Minify data
# 'snap_' columns we can convert to bool or int8
icols = ['event_name_1',
         'event_type_1',
          'event_name_2',
         'event_type_2',
         'snap_CA',
         'snap_TX',
         'snap_WI', 
         'is_first_half_month', 
         'event_bef_weekend', 
         'event_after_weekend',
         'NBA']
for col in icols:
    grid_df[col] = grid_df[col].astype('category')

grid_df['event_attention_after'] = grid_df['event_attention_after'].astype(np.float16) #v2
grid_df['event_attention_bef'] = grid_df['event_attention_bef'].astype(np.float16) #v2 
grid_df['event_attention_sum'] = grid_df['event_attention_sum'].astype(np.float16) #v2 
    
    
    
# Convert to DateTime
grid_df['date'] = pd.to_datetime(grid_df['date'])

# Make some features from date
grid_df['tm_d'] = grid_df['date'].dt.day.astype(np.int8) #day of month
grid_df['tm_w'] = grid_df['date'].dt.week.astype(np.int8) # week of year
grid_df['tm_m'] = grid_df['date'].dt.month.astype(np.int8) # month of year
grid_df['tm_q'] = grid_df['date'].dt.quarter.astype(np.int8) # quarter of year
grid_df['tm_y'] = grid_df['date'].dt.year # year
grid_df['tm_y'] = (grid_df['tm_y'] - grid_df['tm_y'].min()).astype(np.int8) # minus min(year)
grid_df['tm_wm'] = grid_df['tm_d'].apply(lambda x: ceil(x/7)).astype(np.int8) # week of month

grid_df['tm_dw'] = grid_df['date'].dt.dayofweek.astype(np.int8) # day of week: monday start from 0, so saturday. is 5
grid_df['tm_w_end'] = (grid_df['tm_dw']>=5).astype(np.int8) # weekend and weekday

# Remove date
del grid_df['date']

In [55]:
print('Save part 3')

# Safe part 3
grid_df.to_pickle('grid_part_3.pkl')
print('Size:', grid_df.shape)

# We don't need calendar_df anymore
del calendar_df
del grid_df

Save part 3
Size: (46881677, 24)


In [56]:
## Part 1
# Convert 'd' to int
grid_df = pd.read_pickle('grid_part_1.pkl')
grid_df['d'] = grid_df['d'].apply(lambda x: x[2:]).astype(np.int16)

# Remove 'wm_yr_wk'
del grid_df['wm_yr_wk']
grid_df.to_pickle('grid_part_1.pkl')

del grid_df

In [58]:
# add lag feature
grid_df = pd.read_pickle('grid_part_1.pkl')

# We need only 'id','d','sales'
# to make lags and rollings
grid_df = grid_df[['id','d','sales']]
SHIFT_DAY = 28

In [59]:
# Lags
# with 28 day shift
start_time = time.time()
print('Create lags')

LAG_DAYS = [col for col in range(SHIFT_DAY,SHIFT_DAY+15)]
grid_df = grid_df.assign(**{
        '{}_lag_{}'.format(col, l): grid_df.groupby(['id'])[col].transform(lambda x: x.shift(l))
        for l in LAG_DAYS
        for col in [TARGET]
    })

# Minify lag columns
for col in list(grid_df):
    if 'lag' in col:
        grid_df[col] = grid_df[col].astype(np.float16)

print('%0.2f min: Lags' % ((time.time() - start_time) / 60))

# Rollings
# with 28 day shift
start_time = time.time()
print('Create rolling aggs')

for i in [7,14,28,56,168]:
    print('Rolling period:', i)
    grid_df['rolling_mean_'+str(i)] = grid_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(SHIFT_DAY).rolling(i).mean()).astype(np.float16)
    grid_df['rolling_std_'+str(i)]  = grid_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(SHIFT_DAY).rolling(i).std()).astype(np.float16)

# Rollings
# with sliding shift
for d_shift in [1,7,14]: 
    print('Shifting period:', d_shift)
    for d_window in [7,14,28,56]:
        col_name = 'rolling_mean_tmp_'+str(d_shift)+'_'+str(d_window)
        grid_df[col_name] = grid_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(d_shift).rolling(d_window).mean()).astype(np.float16)
    
    
print('%0.2f min: Lags' % ((time.time() - start_time) / 60))

Create lags
5.29 min: Lags
Create rolling aggs
Rolling period: 7
Rolling period: 14
Rolling period: 28
Rolling period: 56
Rolling period: 168
Shifting period: 1
Shifting period: 7
Shifting period: 14
11.73 min: Lags


In [60]:
grid_df[grid_df['id']=='HOBBIES_1_001_CA_1_validation'].iloc[:60]

,id,d,sales,sales_lag_28,sales_lag_29,sales_lag_30,sales_lag_31,sales_lag_32,sales_lag_33,sales_lag_34,...,rolling_mean_tmp_1_28,rolling_mean_tmp_1_56,rolling_mean_tmp_7_7,rolling_mean_tmp_7_14,rolling_mean_tmp_7_28,rolling_mean_tmp_7_56,rolling_mean_tmp_14_7,rolling_mean_tmp_14_14,rolling_mean_tmp_14_28,rolling_mean_tmp_14_56
16838178,HOBBIES_1_001_CA_1_validation,897,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16862809,HOBBIES_1_001_CA_1_validation,898,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16887440,HOBBIES_1_001_CA_1_validation,899,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16912071,HOBBIES_1_001_CA_1_validation,900,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16936702,HOBBIES_1_001_CA_1_validation,901,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16961333,HOBBIES_1_001_CA_1_validation,902,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16985964,HOBBIES_1_001_CA_1_validation,903,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17010595,HOBBIES_1_001_CA_1_validation,904,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17035335,HOBBIES_1_001_CA_1_validation,905,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17060075,HOBBIES_1_001_CA_1_validation,906,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
grid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46881677 entries, 0 to 46881676
Data columns (total 40 columns):
id                        category
d                         int16
sales                     float64
sales_lag_28              float16
sales_lag_29              float16
sales_lag_30              float16
sales_lag_31              float16
sales_lag_32              float16
sales_lag_33              float16
sales_lag_34              float16
sales_lag_35              float16
sales_lag_36              float16
sales_lag_37              float16
sales_lag_38              float16
sales_lag_39              float16
sales_lag_40              float16
sales_lag_41              float16
sales_lag_42              float16
rolling_mean_7            float16
rolling_std_7             float16
rolling_mean_14           float16
rolling_std_14            float16
rolling_mean_28           float16
rolling_std_28            float16
rolling_mean_56           float16
rolling_std_56            float16
ro

In [62]:
print('Save lags and rollings')
grid_df.to_pickle('lags_df_'+str(SHIFT_DAY)+'.pkl')

Save lags and rollings


In [63]:
# group aggregation
grid_df = pd.read_pickle('grid_part_1.pkl')
grid_df[TARGET][grid_df['d']>(1913-28)] = np.nan

# ### V2 ##### 
# grid_df["date"]=[datetime.timedelta(days=x-1)+datetime.datetime(2011,1,29) for x in grid_df["d"]]
# grid_df['month'] = grid_df['date'].dt.month.astype(np.int8) # month of year
# #############


base_cols = list(grid_df)

print(base_cols)




icols =  [
            ['state_id'],
            ['store_id'],
            ['cat_id'],
            ['dept_id'],
            ['state_id', 'cat_id'],
            ['state_id', 'dept_id'],
            ['store_id', 'cat_id'],
            ['store_id', 'dept_id'],
            ['item_id'],
            ['item_id', 'state_id'],
            ['item_id', 'store_id'],
    
### V2 ##### 
#             ['month', 'item_id'],
########   
    
            ]

for col in icols:
    print('Encoding', col)
    col_name = '_'+'_'.join(col)+'_'
    grid_df['enc'+col_name+'mean'] = grid_df.groupby(col)[TARGET].transform('mean').astype(np.float16)
    grid_df['enc'+col_name+'std'] = grid_df.groupby(col)[TARGET].transform('std').astype(np.float16)

    
keep_cols = [col for col in list(grid_df) if col not in base_cols]
grid_df = grid_df[['id','d']+keep_cols]

['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd', 'sales', 'release']
Encoding ['state_id']
Encoding ['store_id']
Encoding ['cat_id']
Encoding ['dept_id']
Encoding ['state_id', 'cat_id']
Encoding ['state_id', 'dept_id']
Encoding ['store_id', 'cat_id']
Encoding ['store_id', 'dept_id']
Encoding ['item_id']
Encoding ['item_id', 'state_id']
Encoding ['item_id', 'store_id']


In [64]:
grid_df.columns

Index(['id', 'd', 'enc_state_id_mean', 'enc_state_id_std', 'enc_store_id_mean',
       'enc_store_id_std', 'enc_cat_id_mean', 'enc_cat_id_std',
       'enc_dept_id_mean', 'enc_dept_id_std', 'enc_state_id_cat_id_mean',
       'enc_state_id_cat_id_std', 'enc_state_id_dept_id_mean',
       'enc_state_id_dept_id_std', 'enc_store_id_cat_id_mean',
       'enc_store_id_cat_id_std', 'enc_store_id_dept_id_mean',
       'enc_store_id_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std',
       'enc_item_id_state_id_mean', 'enc_item_id_state_id_std',
       'enc_item_id_store_id_mean', 'enc_item_id_store_id_std'],
      dtype='object')

In [65]:
grid_df.head()

,id,d,enc_state_id_mean,enc_state_id_std,enc_store_id_mean,enc_store_id_std,enc_cat_id_mean,enc_cat_id_std,enc_dept_id_mean,enc_dept_id_std,...,enc_store_id_cat_id_mean,enc_store_id_cat_id_std,enc_store_id_dept_id_mean,enc_store_id_dept_id_std,enc_item_id_mean,enc_item_id_std,enc_item_id_state_id_mean,enc_item_id_state_id_std,enc_item_id_store_id_mean,enc_item_id_store_id_std
0,HOBBIES_1_008_CA_1_validation,1,1.576172,4.605469,1.639648,4.476562,0.708984,2.259766,0.865234,2.544922,...,1.004883,3.128906,1.261719,3.548828,4.695312,7.183594,6.589844,8.796875,7.214844,9.101562
1,HOBBIES_1_009_CA_1_validation,1,1.576172,4.605469,1.639648,4.476562,0.708984,2.259766,0.865234,2.544922,...,1.004883,3.128906,1.261719,3.548828,0.850098,1.754883,1.133789,2.101562,1.190430,2.019531
2,HOBBIES_1_010_CA_1_validation,1,1.576172,4.605469,1.639648,4.476562,0.708984,2.259766,0.865234,2.544922,...,1.004883,3.128906,1.261719,3.548828,0.611328,0.863281,0.562012,0.827637,0.719727,0.922852
3,HOBBIES_1_012_CA_1_validation,1,1.576172,4.605469,1.639648,4.476562,0.708984,2.259766,0.865234,2.544922,...,1.004883,3.128906,1.261719,3.548828,0.384766,0.692871,0.427490,0.728027,0.397461,0.652344
4,HOBBIES_1_015_CA_1_validation,1,1.576172,4.605469,1.639648,4.476562,0.708984,2.259766,0.865234,2.544922,...,1.004883,3.128906,1.261719,3.548828,4.441406,6.703125,6.960938,8.382812,6.113281,7.390625


In [66]:
print('Save Mean/Std encoding')
grid_df.to_pickle('mean_encoding_df.pkl')

Save Mean/Std encoding
